In [ ]:
# connect gdrive to access inswapper onxx model
from google.colab import drive

print("Connecting to Google Drive...")
# This will prompt you for authorization./
drive.mount('/content/drive')

In [ ]:
import os
import shutil

# This is the path to the file in your Google Drive.
# If you did not put it inside any folders, this path is correct.
drive_file_path = '/content/drive/My Drive/inswapper_128.onnx'

# This is the destination folder the AI script needs.
target_dir = '/root/.insightface/models'
target_path = os.path.join(target_dir, 'inswapper_128.onnx')

print(f"Attempting to copy file from your Drive: {drive_file_path}")

try:
    # Create the destination directory
    os.makedirs(target_dir, exist_ok=True)

    # Copy the file from Google Drive to the Colab environment
    shutil.copy(drive_file_path, target_path)

    print(f"✅ Success! Model has been copied to {target_path}")
    print("\nSetup is complete! You can now run the main Fashion AI script.")

except FileNotFoundError:
    print(f"❌ ERROR: The file was not found at '{drive_file_path}'.")
    print("Please check two things:")
    print("1. That the file is named 'inswapper_128.onnx' in your Drive.")
    print("2. That the file is in your main 'My Drive' area and not inside a folder.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
## libraries to install
!pip install flask pyngrok insightface==0.7.3 diffusers transformers accelerate onnxruntime-gpu --quiet

First paste your hugging face token in secret(key) section
go to hugging face. login go to setting then access token create new token and then go in read then generate token. copy that and paste in secret section in colab.                 
name should HF_TOKEN

In [ ]:
# File: colab_ai_worker.py
# Purpose: Runs in Google Colab to perform heavy AI computation.

# --- 2. IMPORTS ---
import os
import shutil
from google.colab import drive
import cv2
import torch
import numpy as np
from PIL import Image
import insightface
from insightface.app import FaceAnalysis
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from flask import Flask, request, send_file
from pyngrok import ngrok, conf
import io
import logging
import json

# Suppress pyngrok and werkzeug logging to keep the output clean
logging.basicConfig(level=logging.ERROR)
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

# --- 3. SETUP MODEL FILE FROM GOOGLE DRIVE ---
print("Connecting to Google Drive...")
drive.mount('/content/drive')

drive_file_path = '/content/drive/My Drive/inswapper_128.onnx'
target_file_path = 'inswapper_128.onnx'

if os.path.exists(drive_file_path):
    print("Copying 'inswapper_128.onnx' from Google Drive...")
    shutil.copy(drive_file_path, target_file_path)
    print("✅ Model file is ready.")
else:
    raise FileNotFoundError(f"CRITICAL ERROR: The file 'inswapper_128.onnx' was not found in your Google Drive at '{drive_file_path}'")

# --- 4. THE AI LOGIC ---
class FaceConsistentFashionAI:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.face_app = None
        self.fashion_pipeline = None
        self.face_swapper = None
        self.setup_models()

    def setup_models(self):
        print("Setting up face analysis model...")
        self.face_app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        self.face_app.prepare(ctx_id=0, det_size=(640, 640))

        print("Loading face swapper model...")
        self.face_swapper = insightface.model_zoo.get_model(target_file_path, download=False, download_zip=False)

        print("Setting up diffusion model...")
        model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
        self.fashion_pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)

        self.fashion_pipeline.scheduler = DPMSolverMultistepScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            num_train_timesteps=1000,
            trained_betas=None,
            prediction_type="epsilon",
            thresholding=False,
            algorithm_type="sde-dpmsolver++",
            solver_type="midpoint",
            lower_order_final=True,
            use_karras_sigmas=True
        )

        self.fashion_pipeline = self.fashion_pipeline.to(self.device)
        print("✅ All AI models loaded successfully!")

    def get_source_face(self, image_bytes):
        nparr = np.frombuffer(image_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        faces = self.face_app.get(img)
        if not faces: return None
        return sorted(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]), reverse=True)[0]

    def run_generation(self, source_face, height, weight, age, gender, style, occasion, preferred_colors, clothing_items):
        body_description = f"a {age}-year-old {gender}, approximately {height}cm tall and {weight}kg body type"
        clothing_list_str = ", ".join(clothing_items)

        # Shortened prompt to be under the 77-token limit and avoid warnings
        prompt = (f"Full body photo of {body_description}. "
                  f"Wearing a {style} outfit for {occasion}, including a {clothing_list_str}. ")
        if preferred_colors:
            prompt += f"Color palette: {preferred_colors}. "
        prompt += "Professional studio lighting, high fashion, photorealistic, sharp focus, 8k."

        negative_prompt = ("ugly, deformed, disfigured, poor details, bad anatomy, blurry, low quality, "
                           "distorted, extra limbs, missing limbs, multiple people, watermark, text, logo, "
                           "cartoon, anime, painting, drawing, cropped head, out of frame.")
        if gender == "male": negative_prompt += " woman, female, girl, feminine features."
        elif gender == "female": negative_prompt += " man, male, boy, masculine features."

        print(f"\n--- PROMPT ---\n{prompt}\n--------------\n")
        generator = torch.Generator(device=self.device).manual_seed(torch.randint(0, 1000000, (1,)).item())
        base_image = self.fashion_pipeline(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=30,
                                           guidance_scale=7.5, width=512, height=768, generator=generator).images[0]

        generated_img_cv2 = cv2.cvtColor(np.array(base_image), cv2.COLOR_RGB2BGR)

        target_faces = self.face_app.get(generated_img_cv2)
        if target_faces:
            target_face = sorted(target_faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]), reverse=True)[0]
            swapped_image = self.face_swapper.get(generated_img_cv2, target_face, source_face, paste_back=True)
            return Image.fromarray(cv2.cvtColor(swapped_image, cv2.COLOR_BGR2RGB))
        return base_image

# --- 5. FLASK SERVER TO EXPOSE THE AI ---
app = Flask(__name__)
fashion_ai = FaceConsistentFashionAI()

@app.route('/generate', methods=['POST'])
def handle_generation():
    print("\nAI Worker: Received request...")
    if 'user_image' not in request.files: return "Missing 'user_image' file", 400
    try:
        file = request.files['user_image']
        height = request.form.get('height', type=int)
        weight = request.form.get('weight', type=int)
        age = request.form.get('age', type=int)
        gender = request.form.get('gender')
        style = request.form.get('style')
        occasion = request.form.get('occasion')
        preferred_colors = request.form.get('preferred_colors', '')
        clothing_items = json.loads(request.form.get('clothing_items'))

        source_face = fashion_ai.get_source_face(file.read())
        if not source_face: return "Could not detect a face in the uploaded image.", 400

        print(f"AI Worker: Generating image for a {gender}...")
        final_image = fashion_ai.run_generation(source_face, height, weight, age, gender, style, occasion, preferred_colors, clothing_items)

        buf = io.BytesIO()
        final_image.save(buf, format='PNG')
        buf.seek(0)

        print("AI Worker: Sending image back.")
        return send_file(buf, mimetype='image/png')
    except Exception as e:
        print(f"AI Worker ERROR: {e}")
        return str(e), 500

# --- 6. NGROK TUNNEL TO EXPOSE THE SERVER --
-
# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken

conf.get_default().auth_token = "your ngrok token"  # <--- PASTE YOUR NGROK TOKEN HERE
public_url = ngrok.connect(5000)
print("================================================================")
print(f"✅ AI Worker is LIVE at: {public_url}")
print("COPY THIS URL and paste it into your fastapi_backend.py script.")
print("================================================================")

app.run(port=5000)


In [ ]:
# after running above code you get a link like this "https://3431-35-247-122-107.ngrok-free.app"
# paste that link in backend file then run backend after that run frontend